In [1]:
import torch
torch.nn.Module.dump_patches = True
import os
import shutil
import random
import time
import cv2
import numpy as np
from yacs.config import CfgNode as CN
import sys
import torch
import copy
import queue
import glob
import math
import random
import itertools
import threading
from scipy.spatial.distance import pdist

In [2]:
from utils.datasets import LoadStreams,LoadImages
from utils.utils import torch_utils,google_utils,non_max_suppression,Path,scale_coords,plot_one_box,platform,xyxy2xywh

In [3]:
class KalmanFilter(object):
    def __init__(self,bbox):
        (x,y,w,h)=bbox
        self.x,self.y=0,0
        self.last_measurement = self.current_measurement = np.array((2,1),np.float32)
        np_bbox=np.asarray([x,y]).astype(np.float32)
        self.last_predicition = self.current_prediction = np_bbox.resize((2,1))#np.zeros((2,1),np.float32)
        self.kalman = cv2.KalmanFilter(4, 2)
        #设置测量矩阵
        self.kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
        #设置转移矩阵
        self.kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
        #设置过程噪声协方差矩阵
        self.kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32)*0.003
    def move(self,x,y):
        #初始化
        self.last_measurement = self.current_measurement
        self.last_prediction = self.current_prediction
        #传递当前测量坐标值
        self.current_measurement = np.array([[np.float32(x)],[np.float32(y)]])
        #用来修正卡尔曼滤波的预测结果
        self.kalman.correct(self.current_measurement)
        # 调用kalman这个类的predict方法得到状态的预测值矩阵，用来估算目标位置
        current_prediction = self.kalman.predict()
        #上一次测量值
        lmx,lmy = self.last_measurement[0],self.last_measurement[1]
        #当前测量值
        cmx,cmy = self.current_measurement[0],self.current_measurement[1]
        #上一次预测值
        #lpx,lpy = last_prediction[0],last_prediction[1]
        #当前预测值
        cpx,cpy = current_prediction[0],current_prediction[1]
        return cpx,cpy
    def update(self,bbox):
        (x,y,w,h)=bbox
        self.x,self.y = self.move(x,y)
    def get(self):
        return self.x,self.y

In [4]:
import itertools
import numpy as np
from numpy import random
from scipy.optimize import linear_sum_assignment
# 任务分配类
class TaskAssignment(object):         
    # 全排列方法
    def all_permutation(self, task_matrix):
        number_of_choice = len(task_matrix)
        solutions = []
        values = []
        for each_solution in itertools.permutations(range(number_of_choice)):
            each_solution = list(each_solution)
            solution = []
            value = 0
            for i in range(len(task_matrix)):
                value += task_matrix[i][each_solution[i]]
                solution.append(task_matrix[i][each_solution[i]])
            values.append(value)
            solutions.append(solution)
        min_cost = np.min(values)
        best_solution = solutions[values.index(min_cost)]
        return best_solution

    # 匈牙利方法
    def Hungary(self, task_matrix):
        b = task_matrix.copy()
        # 行和列减0
        for i in range(len(b)):
            row_min = np.min(b[i])
            for j in range(len(b[i])):
                b[i][j] -= row_min
        for i in range(len(b[0])):
            col_min = np.min(b[:, i])
            for j in range(len(b)):
                b[j][i] -= col_min
        line_count = 0
        # 线数目小于矩阵长度时，进行循环
        while (line_count < len(b)):
            line_count = 0
            row_zero_count = []
            col_zero_count = []
            for i in range(len(b)):
                row_zero_count.append(np.sum(b[i] == 0))
            for i in range(len(b[0])):
                col_zero_count.append((np.sum(b[:, i] == 0)))
            # 划线的顺序（分行或列）
            line_order = []
            row_or_col = []
            for i in range(len(b[0]), 0, -1):
                while (i in row_zero_count):
                    line_order.append(row_zero_count.index(i))
                    row_or_col.append(0)
                    row_zero_count[row_zero_count.index(i)] = 0
                while (i in col_zero_count):
                    line_order.append(col_zero_count.index(i))
                    row_or_col.append(1)
                    col_zero_count[col_zero_count.index(i)] = 0
            # 画线覆盖0，并得到行减最小值，列加最小值后的矩阵
            delete_count_of_row = []
            delete_count_of_rol = []
            row_and_col = [i for i in range(len(b))]
            for i in range(len(line_order)):
                if row_or_col[i] == 0:
                    delete_count_of_row.append(line_order[i])
                else:
                    delete_count_of_rol.append(line_order[i])
                c = np.delete(b, delete_count_of_row, axis=0)
                c = np.delete(c, delete_count_of_rol, axis=1)
                line_count = len(delete_count_of_row) + len(delete_count_of_rol)
                # 线数目等于矩阵长度时，跳出
                if line_count == len(b):
                    break
                # 判断是否画线覆盖所有0，若覆盖，进行加减操作
                if 0 not in c:
                    row_sub = list(set(row_and_col) - set(delete_count_of_row))
                    min_value = np.min(c)
                    for i in row_sub:
                        b[i] = b[i] - min_value
                    for i in delete_count_of_rol:
                        b[:, i] = b[:, i] + min_value
                    break
        row_ind, col_ind = linear_sum_assignment(b)
        #min_cost = task_matrix[row_ind, col_ind].sum()
        best_solution = list(task_matrix[row_ind, col_ind])
        return  best_solution

In [25]:
from torchvision.models.segmentation import fcn_resnet101
from PIL import Image
from torchvision import transforms
from scipy.optimize import linear_sum_assignment
from extractor.feature_extractor import Extractor
extractor = Extractor("./extractor/checkpoint/ckpt.t7", use_cuda=True)
class PCA:
    def __init__(self,dir_path='./test_colors/train',NUM_EIGEN_FACES = 64):
        self.save={'000':{'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'kalman':[],'cross':False,'overlap':False}}
        self.NUM_EIGEN_FACES = NUM_EIGEN_FACES
        self.images = self.readImages(dir_path)
        self.data = self.createDataMatrix(self.images)
        #self.data = torch.from_numpy(self.data).cuda()
        #torch.pca_lowrank(A, q=None, center=True, niter=2)
        #self.mean, self.eigenVectors = torch.pca_lowrank(self.data, q=None, center=True, niter= self.NUM_EIGEN_FACES)
        #self.means, self.eigenVectorses=[],[]
        #print('the alldata %d'%len(self.eigenVectors))
        #self.init_pca(basepath)
        #self.notebook={'000':{'frame':[],'lib':[]}}
        self.frame=0
        self.color_old=0
        self.flag_frame=[]
        self.flag_line=[]
        self.dth = 0.05#相似度距离0.5~0.6  / 欧氏距离25
        self.Forget=1  #状态切换帧
        self.testsave={}
        self.models_mean = './models/mean.npy'
        self.models_eigenVector = './models/eigenVector.npy'
        #self.meta = threading.Lock()
        if os.path.exists(self.models_mean)==False and os.path.exists(self.models_eigenVector)==False:
            self.mean, self.eigenVectors = cv2.PCACompute(self.data, mean=None, maxComponents=self.NUM_EIGEN_FACES)
            np.save(self.models_mean,self.mean)
            np.save(self.models_eigenVector,self.eigenVectors)
        else:
            self.mean=np.load(self.models_mean)
            self.eigenVectors=np.load(self.models_eigenVector)
        #卡尔曼滤波
        self.kalman=[]
        self.cross_id=[]
        #匈牙利算法
        self.task = TaskAssignment()
    def createDataMatrix(self,images):
        numImages = len(images)
        sz = images[0].shape
        data = np.zeros((numImages, sz[0] * sz[1] * sz[2]), dtype=np.float32)
        for i in range(0, numImages):
            image = images[i].flatten()
            data[i,:] = image
        #print("createData ok")
        return data

    def readImages(self,path):
        print("Reading images from " + path, end="...")
        # Create array of array of images.
        images = []
        # List all files in the directory and read points from text files one by one
        for name in glob.glob(path+'/*'):
            for imagePath in glob.glob(name+'/*.jpg'):
                    # Add to array of images
                    im = cv2.imread(imagePath)
                    im = cv2.resize(im,(64,128))
                    if im is None :
                        print("image:{} not read properly".format(imagePath))
                    else :
                        # Convert image to floating point
                        im = np.float32(im)/255.0
                        # Add image to list
                        images.append(im)
                        # Flip image 
                        imFlip = cv2.flip(im, 1);
                        # Append flipped image
                        #images.append(imFlip)
        numImages = int(len(images))
        # Exit if no image found
        if numImages == 0 :
            print("No images found")
            sys.exit(0)
        print(str(numImages) + " files read.")
        return images
#*
#*
#*      算法集合
#*
#*
    def compute_cos(self,x,y):
        x_,y_=x.flatten(),y.flatten()
        dist =1- abs(np.dot(x_,y_)/(np.linalg.norm(x_)*np.linalg.norm(y_)))      
        return abs(dist)
    
    def compute_dis(self,x,y):
         return np.linalg.norm( x - y )
    
    def compute_Maha(self,x,y):
        X=np.vstack([x,y])
        XT=X.T
        S=np.cov(X)   #两个维度之间协方差矩阵
        SI = np.linalg.inv(S) #协方差矩阵的逆矩阵
        #马氏距离计算两个样本之间的距离，此处共有10个样本，两两组合，共有45个距离。
        n=XT.shape[0]
        d1=[]
        for i in range(0,n):
            for j in range(i+1,n):
                delta=XT[i]-XT[j]
                d=np.sqrt(np.dot(np.dot(delta,SI),delta.T))
                d1.append(d)
        return d1
    
    
     #计算两点近距离公式 xyxy
    def distEclud(self,veA,vecA,veB,vecB):
        lossA=veB-veA
        lossB=vecB-vecA
        return math.sqrt(pow(lossA,2)+pow(lossB,2))
#*
#*
#*      功能函数
#*
#*
    def clear_id(self):
        self.save={'000':{'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'kalman':[],'cross':False,'overlap':False}}  
        
    def kalman_distance(self,bbox,name='cross'):
        (x,y,w,h)=bbox
        list_kalman_distance,list_kalman_id=[],[]
        for k,v in list(self.save.items()):
            if k!='000'and v['passage']==True:
                list_kalman_distance.append(round(self.distEclud(x,y,v['kalman'][0],v['kalman'][1]),3))
                list_kalman_id.append(k)
        return list_kalman_distance,list_kalman_id

    def kalman_bbox_update(self,id_,bbox):
        self.save[id_]['bbox']=bbox
        kalman_id=int(id_)-1
        self.kalman[kalman_id].update(bbox)
        x,y=self.kalman[kalman_id].get()
        self.save[id_]['kalman']=[x,y]
        return (int(x),int(y))
            
    #轨迹跟踪被覆盖的bbox 后得到被覆盖的bbox的预测          
    def kalman_overlap_get(self,id_,bbox):
        (x,y,w,h)=bbox
        bboxs=[]
        for k,v in list(self.save.items()):
            if k!=id_ and k!='000' and v['cross']==True:
                bbox_lib = v['bbox']
                (xa,ya,wa,ha)=bbox_lib
                dis_ = self.distEclud(x,y,xa,ya)
                if (dis_/(w+wa))<0.5:
                    kalman_id=int(k)-1
                    for i in range(3):
                        x,y=self.kalman[kalman_id].get()
                        self.kalman[kalman_id].update([x,y,0,0])
                        x,y=self.kalman[kalman_id].get()
                        self.save[k]['kalman']=[x,y]
                    
                bboxs.append([x,y,wa,ha])   
        return bboxs

    def kalman_reid(self,bbox):
        (x,y,w,h)=bbox
        color_=0
        list_kalman_distance,list_kalman_id=[],[]
        for k,v in list(self.save.items()):
            if k!='000' and v['passage']==True:
                if v['overlap']==True:
                    bbox_ = v['bbox']
                    (xa,ya,wa,ha)=bbox_
                    dis_ = self.distEclud(x,y,xa,ya)
                    if (dis_/(w+wa))<0.5:
                        list_kalman_distance.append(self.distEclud(x,y,v['kalman'][0],v['kalman'][1]))
                        list_kalman_id.append(k)
                elif v['cross']==True:
                    bbox_ = v['bbox']
                    (xa,ya,wa,ha)=bbox_
                    dis_ = self.distEclud(x,y,xa,ya)
                    if (dis_/(w+wa))<0.5:
                        list_kalman_distance.append(self.distEclud(x,y,v['kalman'][0],v['kalman'][1]))
                        list_kalman_id.append(k)
        if len(list_kalman_id)>0:
            id_ = list_kalman_id[list_kalman_distance.index(min(list_kalman_distance))]
            color_=int(len(self.save[id_]['lib'])/2)
            #for cross_id in list_kalman_id:
            #    if cross_id!=id_:
            #        self.cross_off(cross_id)
        else:
            id_ = ' ';color_=0
        return id_,color_

    def manage_lib(self,frame):
        for k,v in list(self.save.items()):
            if k=='000':#or v['passage']==False:
                continue
            lib = v['lib']
            frame_lib=v['frame']
            if frame-frame_lib>self.Forget:
                self.save[k]['frame']=frame
                #self.save[k]['lib'].pop(0)
                #if len(lib)<10:
                    #self.dele_lib_id(k)
                #if  k not in self.save:
                #    continue
                if self.save[k]['passage']==True:
                    center =self.save[k]['center']
                    for lib_ in self.save[k]['lib']:
                        center=center*0.99+lib_*0.01
                    #self.save[k]['lib'].pop(0)
                    self.passage_off(k,center)
                    #self.flag_frame=[]
                    
    def paassag_on(self,feature,bbox,frame,id_):
        self.save[id_]['center']=feature
        self.save[id_]['bbox']=bbox
        self.save[id_]['lib'].append(feature)
        self.save[id_]['frame']=frame
        self.save[id_]['passage']=True
        print('%s passage is True'%id_)
        
    def passage_off(self,id_,center):
        self.save[id_]['center']=center
        self.save[id_]['passage']=False
        print('%s passage is False'%id_)
    
    def overlap_on(self,id_):
        self.save[id_]['overlap']=True
        #print('%s overlap is True'%id_)
        
    def overlap_off(self,id_):
        self.save[id_]['overlap']=False
        #print('%s overlap is False'%id_)
        
    def cross_on(self,id_):
        self.save[id_]['cross']=True
        #print('%s cross is True'%id_)
        
    def cross_off(self,id_):
        self.save[id_]['cross']=False
        #print('%s cross is False'%id_)
        
    def update_frame(self,id_,frame):
        self.save[id_]['frame']=frame
        
    def dele_lib_id(self,id_):
        del self.save[id_]
        print('the id %s Remove'%id_)
        
    def add_lib(self,feature,bbox,frame,passage=False):
        save={'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'kalman':[],'cross':False,'overlap':False}
        for n in  self.save.keys():
            id_=n
        self.kalman.append(KalmanFilter(bbox))
        self.kalman[(int(id_))].update(bbox)
        x,y=self.kalman[(int(id_))].get()
        
        save['kalman']=[x,y]
        self.id_new=('%03d'%(int(id_)+1))
        save['center']=feature
        save['bbox']=bbox
        save['lib']=[feature]
        save['frame']=frame
        save['passage']=passage
        self.save[self.id_new]=save
        print('find id %s'%self.id_new)
        return self.id_new
    
    def add_id(self,feature,bbox,frame):   
        #print(len(self.save))
        if frame!=0 and len(self.save)!=1:
            id_ = self.check_lib_passage(feature)
            #self.id_new = self.check_lib(feature)
            if id_ == ' ':
                self.flag_line=[]
                if self.id_new in self.save:
                    if frame-self.save[self.id_new]['frame']>1:
                        self.flag_frame=[]
                else:
                    self.flag_frame=[]
                if len(self.flag_frame)==0:
                    self.id_new = self.add_lib(feature,bbox,frame)
                self.flag_frame.append(frame)
                self.save[self.id_new]['frame']=frame
                #self.update_frame
                if len(self.flag_frame)>=5:
                    if np.mean(self.flag_frame)==self.flag_frame[2]:
                        self.paassag_on(feature,bbox,frame,self.id_new)
                        self.flag_frame=[]
                    else:
                        self.dele_lib_id(self.id_new)
                        self.flag_frame=[]
            else:
                #if frame - self.save[id_]['frame'] >3:
                self.flag_frame=[]
                self.flag_line.append(frame)
                self.save[id_]['frame']=frame
                #self.update_frame
                if len(self.flag_line)>=3:
                    if np.mean(self.flag_line)==self.flag_line[1]:
                        self.paassag_on(feature,bbox,frame,id_)
                        self.flag_line=[]
                    else:
                        self.flag_line=[]
                return id_
        else:
            self.id_new = self.add_lib(feature,bbox,frame,passage=True)
        return self.id_new
#*
#*      检测集合
#*                        
    def check_frame_id(self,id_,feature,frame):
        list_dis=[]
        for k,v in list(self.save.items()):
            if k==id_:
                if frame-(v['frame'])>=10:
                        if k==id_:
                            for lib_ in v['lib']:
                                list_dis.append(self.compute_cos(lib_,feature))
                            if min(list_dis)<self.dth:
                                return True
                            else:
                                return False
                else:
                    return True
        return False
    #检查除id_me外是否与数据库里的bbox重叠
    def check_bboxs(self,bbox_list,bbox):    
        (xc, yc, wc, hc) =bbox
        list_id,list_x=[],[]
        ret=True
        if len(bbox_list)!=1:
            for i in range(len(bbox_list)):
                for j in range(i+1,len(bbox_list)):
                    (xa, ya, wa, ha) = bbox_list[i]
                    (xb, yb, wb, hb) = bbox_list[j] 
                    distance = self.distEclud(xa,ya,xb,yb)
                    if (distance/(wa+wb))>=0.5:
                        continue
                    if (distance/(wa+wb))<0.5:
                        if xc==xa or xc==xb:#是不是因为重叠导致的
                            ret=False
        return ret
    def check_bboxs_cross(self,bboxs,bbox):    
        ret=True
        flag=False#无交叉但有重叠的标志
        (xa, ya, wa, ha)=bbox
        cross_bboxs=copy.copy(bboxs)
        cross_bboxs.remove(bbox)
        for bbox_ in cross_bboxs:
            (xb, yb, wb, hb) = bbox_ 
            distance = self.distEclud(xa,ya,xb,yb)
            if (distance/(wa+wb))>=0.5:
                continue
            if (distance/(wa+wb))<0.5:
                ret=False
        if self.check_bboxs_overlap(bbox)==False:
            if ret==True:
                flag=True
                ret=False   
        return ret ,flag
    def check_bboxs_overlap(self,bbox):
        ret=True
        flag_num=0
        (xc, yc, wc, hc)=bbox
        for k,v in list(self.save.items()):
            if k!='000'and v['passage']==True:
                (xl, yl, wl, hl) = v['bbox']
                distance = self.distEclud(xc,yc,xl,yl)
                if (distance/(wc+wl))>=0.5:
                    continue
                if (distance/(wc+wl))<0.5:  
                    flag_num+=1
        if flag_num>1:
            ret=False
        return ret
    def check_lib_add(self,featureVector,min_distance_id,bbox):
        for k,v in list(self.save.items()):
            list_dis=[]
            if  k==min_distance_id:
                n=v['center']
                lib = v['lib']
                #跟新kalman 轨迹预测
                kalman_id=int(k)-1
                self.kalman[kalman_id].update(bbox)
                x,y =self.kalman[kalman_id].get()
                self.save[k]['kalman']=[x,y]
                self.cross_off(k)
                self.overlap_off(k)
                for lib_ in lib:
                    list_dis.append(self.compute_cos(lib_,featureVector))
                if len(self.save[min_distance_id]['lib'])>254:
                    num = list_dis.index(min(list_dis))
                    self.save[min_distance_id]['lib'].pop(num)
                    #self.save[min_distance_id]['lib'].pop(0)
                if len(list_dis)>0 and np.mean(list_dis)>self.dth:
                    self.save[min_distance_id]['lib'].append(featureVector)
                    print('\r%s lib len :%d'%(min_distance_id,len(lib)),end='')
                break
        return int(len(lib))
    #检查就绪的
    def check_lib(self,featureVector):
        list_id,list_mean=[],[]
        for k,v in list(self.save.items()):
            list_dis=[]
            if k=='000'or v['passage']==False :
                continue
            lib = v['lib']
            for lib_ in lib:
                list_dis.append(self.compute_cos(lib_,featureVector))
            list_id.append(k)
            list_mean.append(min(list_dis)) 
        if len(list_mean)>0 and min(list_mean) <self.dth:
            return list_id[list_mean.index(min(list_mean))] 
        else:
            #print( min(list_mean))
            return ' '   
    #检查未就绪
    def check_lib_passage(self,featureVector):
        list_id,list_mean=[],[]
        for k,v in list(self.save.items()):
            list_dis=[]
            if k!='000' and v['passage']==False:
                lib = v['lib']
                for lib_ in lib:
                    list_dis.append(self.compute_cos(lib_,featureVector))
                list_id.append(k)
                list_mean.append(min(list_dis)) 
        print(list_mean)
        if len(list_mean)>0 and min(list_mean) <self.dth:
            return list_id[list_mean.index(min(list_mean))] 
        else:
            #print( min(list_mean))
            return ' '    
   
    #def efficientdet_compute_pca(self,bbox_list,confidences,im):
    def cross_video_pca(self,user_bbox_list,confidences,im):
        #
        # * self.notebook 记录发现新的图片的W
        # * 保存数据的格式 self.save={'id':{'center':w,'LIB':[w1..wn]}}
        #
        color=[]
        identities=[]
        dths=[]
        kalman_overlap_bbox=[]
        cross_id =[]
        overlap_id=[]
        guss_bbox=[]
        #维护LIB
        self.manage_lib(self.frame)
        if user_bbox_list!=[]:
            list_id,list_feature,list_distance,list_center=[],[],[],[]
            list_kalman_distance,list_kalman_ids,list_kalman_c,list_kalman_bbox,list_kalman_feature,list_kalman_flag=[],[],[],[],[],[]
            bbox_list=copy.copy(user_bbox_list)
            for bbox,c in zip(user_bbox_list,confidences):
                (x, y, w, h) = bbox
                #print(bbox)
                (x1, y1, x2, y2)=abs(int(x-w/2)), abs(int(y-h/2)), int(x+w/2), int(y+h/2)
                #print((x1, y1, x2, y2))
                im_=im[y1:y2,x1:x2]
                im_ = cv2.resize(im_,(64,128))
                featureVector = self.compute_feature(im_)
                #交叉判断
                #print(bbox_list)
                ret_bbox_cross,flag = self.check_bboxs_cross(user_bbox_list,bbox)
                #print(ret_bbox_cross,flag)
                #ret_bbox_overlap=self.check_bboxs_overlap(bbox_list,bbox)
                #有交叉则使用kalman跟踪
                if ret_bbox_cross ==False :#or ret_bbox_overlap==False:
                    #只是交叉的时候，使用匈牙利算法
                    bbox_list.remove(bbox)
                    if flag==False:
                        list_kalman_distances,list_kalaman_ids = self.kalman_distance(bbox)
                        list_kalman_distance.append(list_kalman_distances)
                        list_kalman_ids.append(list_kalaman_ids)
                        list_kalman_c.append(c)
                        list_kalman_bbox.append(bbox)
                        list_kalman_feature.append(featureVector)
                        list_kalman_flag.append(flag)
                    else:
                        #重叠的时候参考交叉的id情况直接赋予id
                        id_,color_ = self.kalman_reid(bbox)
                        if id_!=' ':
                            #self.cross_on(id_)
                            self.overlap_on(id_)
                            guss_bbox.append(self.kalman_bbox_update(id_,bbox))
                            kalman_overlap_bbox = self.kalman_overlap_get(id_,bbox)
                            identities.append(id_)
                            dths.append(c)
                            color.append(color_)
                            overlap_id.append(id_)
                            
                #PCA算法
                else:
                    distance=0
                #所有图片与LIB中passage为TRUE的计算距离
                    list_distance_buffer,list_id_buffer,list_feature_buffer,list_center_buffer=[],[],[],[]
                    for k,v in list(self.save.items()):
                        if k=='000':
                            continue
                        if v['passage']==True:
                            n=v['center']
                            #kalman_xy=v['kalman']
                            #distance_kalman = self.distEclud(kalman_xy[0],kalman_xy[1],bbox[0],bbox[1])
                            #print(distance_kalman)
                            distance = self.compute_cos(n,featureVector)
                            #distance =+distance_feature*0.95+distance_kalman*0.005
                            list_distance_buffer.append(distance)
                            list_id_buffer.append(k)
                            list_feature_buffer.append(featureVector)
                            list_center_buffer.append(n) 

                    if len(list_distance_buffer)>0 :#and min(list_distance_buffer)<self.dth:
                        list_distance.append(list_distance_buffer)
                        list_id.append(list_id_buffer)
                        list_feature.append(list_feature_buffer)
                        list_center.append(list_center_buffer)
                    else:
                        #ret = self.check_bboxs(bbox_list,bbox) 
                        #if ret ==True:
                        id_ = self.add_id(featureVector,bbox,self.frame)
                        identities.append(id_)
                        dths.append(c)
                        color.append(0) 
            
            #匈牙利算法
            #Kalman 
            if list_kalman_distance!=[]:
                list_kalman_id_=[]
                if len(list_kalman_distance)>len(list_kalman_distance[0]):
                        min_distance=[]
                        #print(list_distance,bbox_list)
                        for kalman_distance in list_kalman_distance:
                            min_distance.append(min(kalman_distance))
                        for i in range((len(list_kalman_distance)-len(list_kalman_distance[0]))):
                            num=min_distance.index(max(min_distance))
                            list_kalman_distance.pop(num)
                            min_distance.pop(num)
                            list_kalman_ids.pop(num)
                #print('data:',len(list_kalman_ids),len(list_kalman_c),len(list_kalman_bbox),len(list_kalman_feature))
                #print(list_kalman_c)
                for id_s in list_kalman_ids:
                    for id_ in id_s:
                        list_kalman_id_.append(id_)
                #print(list_kalman_distance,list_kalman_bbox,list_kalman_id_,list_kalman_c,list_kalman_feature)
                np_kalman_distance = np.asarray(list_kalman_distance)
                #print (np_kalman_distance.shape)
                if np_kalman_distance.shape[0]<2:
                    list_Hungary_kalman=[min(np_kalman_distance.flatten().tolist())]
                else:
                    list_Hungary_kalman = self.task.Hungary(np_kalman_distance)#all_permutation
                #print(np_kalman_distance,list_Hungary_kalman)
                count=0
                for dis in list_Hungary_kalman:
                    number = np_kalman_distance.flatten().tolist().index(dis)
                    identities.append(list_kalman_id_[number])
                    self.save[list_kalman_id_[number]]['bbox'] =list_kalman_bbox[count]
                    self.save[list_kalman_id_[number]]['frame'] =self.frame
                    dths.append(list_kalman_c[count])
                    #color_=self.check_lib_add(list_kalman_feature[count],list_kalman_id_[number],list_kalman_bbox[count])
                    color.append(0)
                    count+=1
                for flag,id_,bbox in zip(list_kalman_flag,identities,list_kalman_bbox):
                    self.cross_on(id_)
                    self.overlap_off(id_)
                    self.kalman_bbox_update(id_,bbox)
                cross_id=identities
                    
            #PCA
            list_bbox_loss,list_feature_loss=[],[]
            if bbox_list!=[] and list_distance!=[]:
                    kalman_center,kalman_feature,kalman_id,kalman_bbox=[],[],[],[]
                    kalman_bbox=bbox_list
                    #需要id的数量要小于等于就绪id的数量
                    #print(len(bbox_list),len(list_distance[0]))
                    if len(bbox_list)>len(list_distance[0]):
                        min_distance=[]
                        #print(list_distance,bbox_list)
                        for id_distanc in list_distance:
                            min_distance.append(min(id_distanc))
                        for i in range((len(bbox_list)-len(list_distance[0]))):
                            num=min_distance.index(max(min_distance))
                            min_distance.pop(num)
                            list_distance.pop(num)
                            list_id.pop(num)
                            list_center.pop(num)  
                            list_feature_loss.append(list_feature.pop(num))
                            list_bbox_loss.append(kalman_bbox.pop(num))
                            #print(list_bbox_loss,list_feature_loss)
                            confidences.pop(num)
                    if list_distance==[]:
                        return identities,dths,color,kalman_overlap_bbox,cross_id,overlap_id,guss_bbox
                    np_distance=np.asarray(list_distance)
                    #print(np_distance)
                    #print(self.save.keys())
                    if np_distance.shape[0]<2:
                        list_Hungary=[min(np_distance.flatten().tolist())]
                    else:
                        list_Hungary = self.task.Hungary(np_distance)#all_permutation
                    #list_Hungary = self.task.Hungary(np_distance)#all_permutation
                    #list_Hungary = self.computer_Hungary(np_distance)
                    #print(np_distance,list_Hungary)
                    list_id_,list_center_,list_feature_=[],[],[]
                    for id_s,center_s,feature_s in zip(list_id,list_center,list_feature):
                        for id_,center_,feature_ in zip(id_s,center_s,feature_s):
                            list_id_.append(id_)
                            list_center_.append(center_)
                            list_feature_.append(feature_)
                    for number in list_Hungary:
                        #number = number*0.00001               
                        min_distance = np_distance.flatten().tolist().index(number)
                        kalman_id.append(list_id_[min_distance])
                        kalman_center.append(list_center_[min_distance])
                        kalman_feature.append(list_feature_[min_distance])

                    #检查重新回来的id的frame
                    #ret_id_frame = self.check_frame_id(min_distance_id,min_distance_feature,self.frame)
                    #print(kalman_bbox,kalman_center,kalman_feature,kalman_id,confidences)
                    #ret_bbox_cross = self.check_bboxs_cross(kalman_bbox,kalman_id)
                    #if ret_bbox_cross==False :
                        #kalman_bbox=self.kalman_reid(kalman_bbox,kalman_id)
                    for bbox,conter,feature,id_,c in zip(kalman_bbox,kalman_center,kalman_feature,kalman_id,confidences):
                         #检查除id_me外是否与数据库里的bbox重叠
                        center_ = (conter*0.5)+(feature*0.5)
                        self.save[id_]['center'] =center_
                        self.save[id_]['bbox'] =bbox
                        self.save[id_]['frame'] =self.frame
                        color_=self.check_lib_add(feature,id_,bbox)
                        color.append(color_)
                        dths.append(c)
                        identities.append(id_)
                    #没有ID,重新add_id    
                    #print(list_bbox_loss,list_feature_loss,confidences)
                    for bbox,featureVector,c in zip(list_bbox_loss,list_feature_loss,confidences):
                        #print(bbox,featureVector,c)
                        id_ = self.add_id(featureVector[0],bbox,self.frame)
                        identities.append(id_)
                        dths.append(c)
                        color.append(0)  
        self.frame+=1
        #print(identities,dths,color)
        return identities,dths,color ,kalman_overlap_bbox,cross_id,overlap_id,guss_bbox
    #输入图片计算，输出Feature
    def compute_feature(self,im):
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        #print(im.shape)
        feature = extractor([im])
        #print(feature.shape)
        return feature
    
    '''
    def compute_feature(self,im):
        im_ = im
        size=im_.shape
        im_ = np.float32(im_)/255.0
        Fim = im_.flatten()
        Fmean = self.mean.reshape(Fim.shape)
        Fdf = Fim-Fmean
        Fdf= Fdf.reshape(24576,1)
        W=[]
        for i in range(len(self.eigenVectors)):
            E = self.eigenVectors[i,:].reshape(1,24576)
            W.append(np.dot(E,Fdf).flatten())
        W = np.asarray(W)
        return W
    '''
    

In [26]:
def display(bbox_list,c_list,im,identities,dths,color,kalman,t,cross_id,overlap_id,guss_bbox):
    w_im,h_im=im.shape[:2]
    #print(len(out_list))
    for bbox,c,id_,dth,color_ in zip(bbox_list,c_list,identities,dths,color):
        #(x1, y1, x2, y2) = bbox
        (x, y, w, h) = bbox
        (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
        cv2.rectangle(im, (x1, y1), (x2, y2), (0, color_, 255-color_), 2)
        #cv2.putText(im, ('%.2f'%c), (x2+10, y1+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 250,0), 1)
        cv2.putText(im, ('%s'%id_), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, color_, 255-color_), 2)
        cv2.putText(im, ('%.2f'%dth), (x2, y1+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, color_, 255-color_), 2)
        cv2.circle(im, (int(x),int(y)), 1, (0, 255, 0), 2)
    num=0
    for xy in guss_bbox:
        cv2.circle(im, xy, 1, (0, 0, 255), 2)
    for cid_ in cross_id:
        cv2.putText(im, ('Crossid : %s'%cid_), (w_im, 100+num), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        num+=50
    for oid_ in overlap_id:
        cv2.putText(im, ('Overlapid : %s'%oid_), (w_im, 100+num), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        num+=50

    for ka in kalman:  
        (x,y,w,h)=ka
        (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
        cv2.rectangle(im, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.circle(im, (x,y), 1, (0, 0, 255), 2)
    fps = 1/(time.time()-t)
    cv2.putText(im, ('fps:%.2f'%fps), (w_im, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.putText(im, ('Id number: %d'%len(bbox_list)), (w_im, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.imshow('V5', im)
    return im
def yolov5_detect(YOLOV5_CFG,pca):
    device = torch_utils.select_device(YOLOV5_CFG.device)
    model = torch.load(YOLOV5_CFG.weights, map_location=device)['model']
    #model = torch.load(YOLOV5_CFG.weights, map_location=device)
    model.to(device).eval()
    names = model.names if hasattr(model, 'names') else model.modules.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]
    
    if os.path.isfile(YOLOV5_CFG.source) is True:
        cap = cv2.VideoCapture(YOLOV5_CFG.source)
        w0,h0,fps,max_index = int(cap.get(3)),int(cap.get(4)),int(cap.get(5)),int(cap.get(7))
        [w1,h1] = YOLOV5_CFG.im_size
        if YOLOV5_CFG.output is not False:
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
            out = cv2.VideoWriter(YOLOV5_CFG.output, fourcc, fps, (w0,h0))
        for i in range(max_index):
            re,im = cap.read()
            net_input = np.transpose(cv2.resize(im, (w1,h1))/255,(2,0,1)).reshape((-1,3,h1,w1))
            net_input = torch.from_numpy(net_input).to(device).type(torch.float32)
            t = time.time()
            pred = model(net_input, augment=YOLOV5_CFG.augment)[0]
            pred = non_max_suppression(pred, YOLOV5_CFG.conf_thres, YOLOV5_CFG.iou_thres, fast=True, classes=YOLOV5_CFG.classes, agnostic=YOLOV5_CFG.agnostic_nms)
            if pred is None or pred[0] is None:
                cv2.imshow('V5',im)
                if YOLOV5_CFG.output is not False:
                    out.write(im)
                continue
            w_im,h_im=im.shape[:2]
            #print(w_im,h_im)
            area_im=int(w_im*h_im)
            bboxes,confs,cats = pred[0][:,:4].cpu().detach().numpy(),pred[0][:,4].cpu().detach().numpy(),pred[0][:,5].cpu().detach().numpy()
            bboxes[:,[0,2]],bboxes[:,[1,3]]=bboxes[:,[0,2]]*(w0/w1),bboxes[:,[1,3]]*(h0/h1)
            list_bbox,list_conf=[],[]
            for bbox,conf,cat in zip(bboxes.astype(np.int),confs,cats.astype(np.int)):
                if (names[cat]=='person'):
                    #p_min,p_max = (bbox[0],bbox[1]),(bbox[2],bbox[3])
                    #im = cv2.rectangle(im, p_min, p_max, (255,0,123), 1, cv2.LINE_AA)
                    #im = cv2.putText(im, '%s %.2f'%(names[cat],conf), p_min, cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
                    (x1, y1, x2, y2)=bbox
                    bbox_=(int(abs((x1+x2)*0.5)),int(abs((y1+y2)*0.5)),int(abs(x1-x2)),int(abs(y1-y2)))
                    (x,y,w,h)=bbox_
                    area=int(w*h)
                    if (area/area_im) <0.005:
                        continue
                    list_bbox.append(bbox_)
                    list_conf.append(conf)
            #CrossVideo Reid
            identities,dths,color ,kalman,cross_id,overlap_id,guss_bbox = pca.cross_video_pca(list_bbox,list_conf,im)
            display(list_bbox,list_conf,im,identities,dths,color,kalman,t,cross_id,overlap_id,guss_bbox)
            t=time.time()
            if YOLOV5_CFG.output is not False:
                out.write(im)
            if cv2.waitKey(1)&0xff==ord('q'):
                break
        cap.release()
        if YOLOV5_CFG.output is not False:
            out.release()
        cv2.destroyAllWindows()

In [27]:
YOLOV5_CFG = CN()
YOLOV5_CFG.agnostic_nms = False
YOLOV5_CFG.augment      = False
YOLOV5_CFG.classes      = False
YOLOV5_CFG.device       = '0'

YOLOV5_CFG.weights = 'weights/yolov5l.pt'
YOLOV5_CFG.source = './testvideo/threecrossrotate.mp4'
name = time.strftime('%Y.%m.%d',time.localtime(time.time()))
YOLOV5_CFG.output = './savevideo/'+name+'.mp4'


YOLOV5_CFG.save_npz     = False
YOLOV5_CFG.conf_thres   = 0.3
YOLOV5_CFG.iou_thres    = 0.3
YOLOV5_CFG.im_size      = [640,512]
YOLOV5_CFG.freeze()

pca=PCA('./pcadata/train2020',256)
yolov5_detect(YOLOV5_CFG,pca)

Reading images from ./pcadata/train2020...1982 files read.
Using CUDA device0 _CudaDeviceProperties(name='GeForce GTX 1080 Ti', total_memory=11264MB)

find id 001
001 passage is False
[0.03786855936050415]
[0.03865402936935425]
[0.035033583641052246]
[0.032151758670806885]
[0.033841073513031006]
[0.031118929386138916]
001 passage is True
[]
find id 002
[0.03248244524002075]
001 passage is False
[0.027536869049072266, 0.04382121562957764]
[0.023078322410583496, 0.043981730937957764]
[0.0211641788482666, 0.042613685131073]
[0.023404240608215332, 0.04310798645019531]
[0.024598896503448486, 0.04289764165878296]
001 passage is True
001 passage is False
[0.03463470935821533, 0.04754769802093506]
[0.03526341915130615, 0.043645620346069336]
[0.03221839666366577, 0.0449865460395813]
[0.029484868049621582, 0.046312689781188965]
[0.02910667657852173, 0.04536384344100952]
[0.030738532543182373, 0.05011337995529175]
001 passage is True
[0.04218423366546631]
[0.0402563214302063]
[0.04704469442367554